In [1]:
!pip3 install --pre deepchem
import deepchem

In [2]:
!pip3 install -q mdtraj nglview

  ERROR: Command errored out with exit status 1:
   command: /Users/i340019/opt/anaconda3/bin/python /Users/i340019/opt/anaconda3/lib/python3.9/site-packages/pip/_vendor/pep517/in_process/_in_process.py build_wheel /var/folders/nm/8ts6zcds2nd18q6vxyzdjd6c0000gn/T/tmpyukf4w3o
       cwd: /private/var/folders/nm/8ts6zcds2nd18q6vxyzdjd6c0000gn/T/pip-install-vexq11m3/mdtraj_3fc9af70c94b4a73903b97618bcdfe9f
  Complete output (661 lines):
  Apple clang version 13.1.6 (clang-1316.0.21.2)
  Target: x86_64-apple-darwin21.3.0
  Thread model: posix
  InstalledDir: /Library/Developer/CommandLineTools/usr/bin
  C compiler:
  Attempting to autodetect OpenMP support... Did not detect OpenMP support
  Attempting to autodetect SSE3   support... Compiler supports SSE3
  Attempting to autodetect SSE4.1 support... Compiler supports SSE4.1
  Attempting to autodetect NEON   support... Did not detect NEON support
  
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build/li

In [3]:
import os
import numpy as np
import pandas as pd

import tempfile

from rdkit import Chem
from rdkit.Chem import AllChem
import deepchem as dc

from deepchem.utils import download_url, load_from_disk

To illustrate the docking procedure, here we'll use a csv that contains SMILES strings of ligands as well as PDB files for the ligand and protein targets from PDBbind. Later, we'll use the labels to train a model to predict binding affinities. We'll also show how to download and featurize PDBbind to train a model from scratch.



In [4]:
data_dir = dc.utils.get_data_dir()
dataset_file = os.path.join(data_dir, "pdbbind_core_df.csv.gz")

if not os.path.exists(dataset_file):
    print('File does not exist. Downloading file...')
    download_url("https://s3-us-west-1.amazonaws.com/deepchem.io/datasets/pdbbind_core_df.csv.gz")
    print('File downloaded...')

raw_dataset = load_from_disk(dataset_file)
raw_dataset = raw_dataset[['pdb_id', 'smiles', 'label']]

In [5]:
raw_dataset

,pdb_id,smiles,label
0,2d3u,CC1CCCCC1S(O)(O)NC1CC(C2CCC(CN)CC2)SC1C(O)O,6.92
1,3cyx,CC(C)(C)NC(O)C1CC2CCCCC2C[NH+]1CC(O)C(CC1CCCCC...,8.00
2,3uo4,OC(O)C1CCC(NC2NCCC(NC3CCCCC3C3CCCCC3)N2)CC1,6.52
3,1p1q,CC1ONC(O)C1CC([NH3+])C(O)O,4.89
4,3ag9,NC(O)C(CCC[NH2+]C([NH3+])[NH3+])NC(O)C(CCC[NH2...,8.05
...,...,...,...
188,2x0y,CN1C2NCN(CC(O)CO)C2C(O)N(C)C1O,4.60
189,3uex,CCCCCCCCCCCCCCCCCC(O)O,6.92
190,2pq9,[O-]C(O)C1CC(OC(O[PH](O)(O)O)(C([O-])O)C(F)F)C...,8.11
191,1u1b,CC1CN(C2CC(O[PH](O)(O)O[PH](O)(O)OCC3OC(N4CNC5...,7.80


In [6]:
from simtk.openmm.app import PDBFile
from pdbfixer import PDBFixer

from deepchem.utils.vina_utils import prepare_inputs

In [7]:
# consider one protein-ligand complex for visualization
pdbid = raw_dataset['pdb_id'].iloc[1]
ligand = raw_dataset['smiles'].iloc[1]

In [8]:
%%time
fixer = PDBFixer(pdbid=pdbid)
PDBFile.writeFile(fixer.topology, fixer.positions, open('%s.pdb' % (pdbid), 'w'))

p, m = None, None
# fix protein, optimize ligand geometry, and sanitize molecules
try:
    p, m = prepare_inputs('%s.pdb' % (pdbid), ligand)
except:
    print('%s failed PDB fixing' % (pdbid)) 

if p and m:  # protein and molecule are readable by RDKit
    print(pdbid, p.GetNumAtoms())
    Chem.rdmolfiles.MolToPDBFile(p, '%s.pdb' % (pdbid))
    Chem.rdmolfiles.MolToPDBFile(m, 'ligand_%s.pdb' % (pdbid))

<timed exec>:7: DeprecationWarning: Call to deprecated function prepare_inputs. Please use the corresponding function in deepchem.utils.docking_utils.


3cyx 1510
CPU times: user 2.24 s, sys: 171 ms, total: 2.41 s
Wall time: 6.65 s


# Visualization

In [ ]:
#!jupyter-nbextension enable nglview --py --sys-prefix
#!jupyter labextension install  nglview-js-widgets 
!pip3 install mdtraj

In [9]:
import mdtraj as md
import nglview

from IPython.display import display, Image

In [11]:
protein_mdtraj = md.load_pdb('3cyx.pdb')
ligand_mdtraj = md.load_pdb('ligand_3cyx.pdb')

In [12]:
v = nglview.show_mdtraj(ligand_mdtraj)
v

NGLWidget()

In [13]:
view = nglview.show_mdtraj(protein_mdtraj)
display(view)

NGLWidget()

In [15]:
import nglview
view = nglview.show_pdbid("1m1n")  # load "3pqr" from RCSB PDB and display viewer widget
view

NGLWidget()